In [1]:
import pandas as pd
import numpy as np

import decoupler as dc
import matplotlib.pyplot as plt

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [19]:
print(dc.__version__)

1.2.0


In [7]:
!wget 'https://zenodo.org/record/7035528/files/knockTF_expr.csv?download=1' -O /Users/smuellerdott/Documents/NTNUdecoupleR/data/knockTF_expr.csv
!wget 'https://zenodo.org/record/7035528/files/knockTF_meta.csv?download=1' -O /Users/smuellerdott/Documents/NTNUdecoupleR/data/knockTF_meta.csv

--2022-10-04 11:48:29--  https://zenodo.org/record/7035528/files/knockTF_expr.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146086808 (139M) [text/plain]
Saving to: ‘/Users/smuellerdott/Documents/NTNUdecoupleR/data/knockTF_expr.csv’

/Users/smuellerdott 100%[===================>] 139,32M  28,0MB/s    in 5,7s    

2022-10-04 11:48:35 (24,4 MB/s) - ‘/Users/smuellerdott/Documents/NTNUdecoupleR/data/knockTF_expr.csv’ saved [146086808/146086808]

--2022-10-04 11:48:36--  https://zenodo.org/record/7035528/files/knockTF_meta.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144861 (141K) [text/plain]
Saving to: ‘/Users/smuellerdott/Documents/NTNUdecoupleR/data/knockTF_meta.csv’

/Users/smuellerdott 100

In [5]:
mat = pd.read_csv('../../data/knockTF_expr.csv', index_col=0)
obs = pd.read_csv('../../data/knockTF_meta.csv', index_col=0)

In [6]:
msk = obs['logFC'] < -1
mat = mat[msk]
obs = obs[msk]
mat.shape, obs.shape, pd.unique(obs['TF'].values).shape

((388, 21985), (388, 13), (234,))

In [7]:
doro_ABC = pd.read_csv('../../data/networks/filtered_dorothea_ABC.csv')
regnet = pd.read_csv('../../data/networks/filtered_regnetwork.csv')
pathComp = pd.read_csv('../../data/networks/filtered_pathwayCommons.csv')
chea3 = pd.read_csv('../../data/networks/filtered_chea3.csv')
collecTRI = pd.read_csv('../../output/CollecTRI/CollecTRI.csv')
collecTRI_rand = dc.shuffle_net(collecTRI, target='target', weight='weight').drop_duplicates(['source', 'target'])

In [8]:
chea3_archs4 = chea3[chea3['confidence'] == 'ARCHS4_Coexpression']
chea3_encode = chea3[chea3['confidence'] == 'ENCODE_ChIP-seq']
chea3_enrich = chea3[chea3['confidence'] == 'Enrichr_Queries']
chea3_GTEx = chea3[chea3['confidence'] == 'GTEx_Coexpression']
chea3_lit = chea3[chea3['confidence'] == 'Literature_ChIP-seq']
chea3_remap = chea3[chea3['confidence'] == 'ReMap_ChIP-seq']

In [9]:
# Remove correlated sources from pathComp
decouple_kws =  {'source': 'source', 'target': 'target', 'weight': 'weight', 'min_n': 5}
mat_pathComp, obs_pathComp, var_pathComp, pathComp, groupby_pathComp = dc.format_benchmark_inputs(mat = mat, obs = obs, sign = -1, net = pathComp, by = 'experiment', perturb='TF', groupby = None, decouple_kws=decouple_kws)

corr_res = dc.check_corr(pathComp)

In [10]:
idx_corr = corr_res['corr'] >= 0.95
corr_tfs = corr_res[idx_corr]

idx_tfs = np.isin(corr_tfs['source2'].values, obs['TF'].values)
tfs_to_remove1 = corr_tfs[idx_tfs]['source1']
tfs_to_remove2 = corr_tfs[~idx_tfs]['source2']

tfs_to_remove = pd.concat([tfs_to_remove1, tfs_to_remove2])

np.sum(np.isin(tfs_to_remove.values, obs['TF'].values))

0

In [11]:
idx_remove = np.isin(pathComp['source'].values, tfs_to_remove.values)
pathComp_filtered = pathComp.loc[~idx_remove]

In [12]:
# Remove correlated sources from regnet
decouple_kws =  {'source': 'source', 'target': 'target', 'weight': 'weight', 'min_n': 5}
mat_regnet, obs_regnet, var_regnet, regnet, groupby_regnet = dc.format_benchmark_inputs(mat = mat, obs = obs, sign = -1, net = regnet, by = 'experiment', perturb='TF', groupby = None, decouple_kws=decouple_kws)

corr_res = dc.check_corr(regnet)

In [13]:
idx_corr = corr_res['corr'] >= 0.95
corr_tfs = corr_res[idx_corr]

idx_tfs = np.isin(corr_tfs['source2'].values, obs['TF'].values)
tfs_to_remove1 = corr_tfs[idx_tfs]['source1']
tfs_to_remove2 = corr_tfs[~idx_tfs]['source2']

tfs_to_remove = pd.concat([tfs_to_remove1, tfs_to_remove2])

np.sum(np.isin(tfs_to_remove.values, obs['TF'].values))

1

In [14]:
idx_remove = np.isin(regnet['source'].values, tfs_to_remove.values)
regnet_filtered = regnet.loc[~idx_remove]

In [15]:
# Number of TFs included in each network
n_doro = np.sum(np.isin(pd.unique(doro_ABC['source'].values), pd.unique(obs['TF'].values))) 
n_collectri = np.sum(np.isin(pd.unique(collecTRI['source'].values), pd.unique(obs['TF'].values)))
n_regnet = np.sum(np.isin(pd.unique(regnet_filtered['source'].values), pd.unique(obs['TF'].values))) 
n_pathComp = np.sum(np.isin(pd.unique(pathComp_filtered['source'].values), pd.unique(obs['TF'].values)))
n_cheaArch = np.sum(np.isin(pd.unique(chea3_archs4['source'].values), pd.unique(obs['TF'].values))) 
n_cheaEncode = np.sum(np.isin(pd.unique(chea3_encode['source'].values), pd.unique(obs['TF'].values)))
n_cheaEnrich = np.sum(np.isin(pd.unique(chea3_enrich['source'].values), pd.unique(obs['TF'].values))) 
n_cheaGTEx = np.sum(np.isin(pd.unique(chea3_GTEx['source'].values), pd.unique(obs['TF'].values)))
n_chealit = np.sum(np.isin(pd.unique(chea3_lit['source'].values), pd.unique(obs['TF'].values))) 
n_cheaRemap = np.sum(np.isin(pd.unique(chea3_remap['source'].values), pd.unique(obs['TF'].values)))
n_doro, n_collectri, n_regnet, n_pathComp, n_cheaArch, n_cheaEncode, n_cheaEnrich, n_cheaGTEx, n_chealit, n_cheaRemap

(125, 171, 123, 92, 156, 46, 146, 155, 66, 101)

In [16]:
# Build dictionary of networks to test
nets = {
    'ABC': doro_ABC,
    'collecTRI': collecTRI,
    'regnet': regnet_filtered,
    'pathComp': pathComp_filtered,
    'chea3_archs4': chea3_archs4,
    'chea3_encode': chea3_encode,
    'chea3_enrich': chea3_enrich,
    'chea3_GTEx': chea3_GTEx,
    'chea3_lit': chea3_lit,
    'chea3_remap': chea3_remap,
    'rand': collecTRI_rand
}

# Example extra arguments
decouple_kws = {
    'ABC': {'args' : {'wsum' : {'times': 1000}}},
    'collecTRI': {'args' : {'wsum' : {'times': 1000}}},
    'regnet': {'args' : {'wsum' : {'times': 1000}}},
    'pathComp': {'args' : {'wsum' : {'times': 1000}}},
    'chea3_archs4': {'args' : {'wsum' : {'times': 1000}}},
    'chea3_encode': {'args' : {'wsum' : {'times': 1000}}},
    'chea3_enrich': {'args' : {'wsum' : {'times': 1000}}},
    'chea3_GTEx': {'args' : {'wsum' : {'times': 1000}}},
    'chea3_lit': {'args' : {'wsum' : {'times': 1000}}},
    'chea3_remap': {'args' : {'wsum' : {'times': 1000}}},
    'rand': {'args' : {'wsum' : {'times': 1000}}}

}

# Run benchmark pipeline
df = dc.benchmark(mat, obs, nets, perturb='TF', sign=-1, verbose=True, decouple_kws=decouple_kws)

Using ABC network...
Extracting inputs...
Formating net...
174 experiments without sources in net, they will be removed.
Running methods...
55 features of mat are empty, they will be removed.
Running mlm on mat with 214 samples and 21930 targets for 297 sources.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.85it/s]


55 features of mat are empty, they will be removed.
Running ulm on mat with 214 samples and 21930 targets for 297 sources.
55 features of mat are empty, they will be removed.
Running wsum on mat with 214 samples and 21930 targets for 297 sources.
Infering activities on 1 batches.


100%|█████████████████████████████████████████████| 1/1 [00:28<00:00, 28.63s/it]


Calculating metrics...
Computing metrics...
Done.
Using collecTRI network...
Extracting inputs...
Formating net...
109 experiments without sources in net, they will be removed.
Running methods...
52 features of mat are empty, they will be removed.
Running mlm on mat with 279 samples and 21933 targets for 774 sources.


...

Calculating metrics...
Computing metrics...
Done.
Using rand network...
Extracting inputs...
Formating net...
109 experiments without sources in net, they will be removed.
Running methods...
52 features of mat are empty, they will be removed.
Running mlm on mat with 279 samples and 21933 targets for 775 sources.


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.32s/it]


52 features of mat are empty, they will be removed.
Running ulm on mat with 279 samples and 21933 targets for 775 sources.
52 features of mat are empty, they will be removed.
Running wsum on mat with 279 samples and 21933 targets for 775 sources.
Infering activities on 1 batches.


100%|█████████████████████████████████████████████| 1/1 [01:08<00:00, 68.82s/it]


Calculating metrics...
Computing metrics...
Done.


In [17]:
pd.DataFrame.to_csv(df, '../../output/benchmark/benchmark_res.csv')

In [18]:
nets = {
    'ABC': doro_ABC,
    'regnet': regnet_filtered,
    'collecTRI': collecTRI
}

# Example extra arguments
decouple_kws = {
    'ABC': {'args' : {'wsum' : {'times': 100}}},
    'regnet': {'args' : {'wsum' : {'times': 100}}},
    'collecTRI': {'args' : {'wsum' : {'times': 100}}}
}

# Run benchmark pipeline
df_source = dc.benchmark(mat, obs, nets, perturb='TF', sign=-1, by='source', verbose=True, decouple_kws=decouple_kws)

Using ABC network...
Extracting inputs...
Formating net...
174 experiments without sources in net, they will be removed.
Running methods...
55 features of mat are empty, they will be removed.
Running mlm on mat with 214 samples and 21930 targets for 297 sources.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.88it/s]


55 features of mat are empty, they will be removed.
Running ulm on mat with 214 samples and 21930 targets for 297 sources.
55 features of mat are empty, they will be removed.
Running wsum on mat with 214 samples and 21930 targets for 297 sources.
Infering activities on 1 batches.


100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.16s/it]


Calculating metrics...
Computing metrics...
Done.
Using regnet network...
Extracting inputs...
Formating net...
159 experiments without sources in net, they will be removed.
Running methods...
54 features of mat are empty, they will be removed.
Running mlm on mat with 229 samples and 21931 targets for 622 sources.


...

Calculating metrics...
Computing metrics...
Done.
Using collecTRI network...
Extracting inputs...
Formating net...
109 experiments without sources in net, they will be removed.
Running methods...
52 features of mat are empty, they will be removed.
Running mlm on mat with 279 samples and 21933 targets for 774 sources.


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.31s/it]


52 features of mat are empty, they will be removed.
Running ulm on mat with 279 samples and 21933 targets for 774 sources.
52 features of mat are empty, they will be removed.
Running wsum on mat with 279 samples and 21933 targets for 774 sources.
Infering activities on 1 batches.


100%|█████████████████████████████████████████████| 1/1 [00:07<00:00,  7.07s/it]


Calculating metrics...
Computing metrics...
Done.


In [19]:
pd.DataFrame.to_csv(df_source, '../../output/benchmark/benchmark_source_res.csv')